### 캐글 산탄데르 은행 금융상품 추천 모델링 - Baseline model 사례 연구

### Baseline model이란?

- Baseline model은 일반적인 머신러닝 파이프라인의 모든 과정을 포함하는 가장 기초적 모델
- 머신러닝 파이프라인의 일반적인 순서  
  1. 데이터 전처리
  2. 피처 엔지니어링
  3. 머신러닝 모델 학습
  4. 테스트 데이터 예측 및 캐글 업로드

<img src='images/data_flow.jpg'>

In [1]:
import pandas as pd
import numpy as np

# !pip install xgboost
import xgboost as xgb

from pytz import timezone
import datetime

In [2]:
# 집계 함수
def value_counts_add_percent(ser):
    df = pd.DataFrame({'count':ser.value_counts(dropna=False),
                     'percent':ser.value_counts(dropna=False) / len(ser) * 100})
    return df

In [3]:
# 데이터를 불러온다. 
np.random.seed(2018)

trn = pd.read_csv('train_ver2.csv')
tst = pd.read_csv('test_ver2.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
# 데이터 크기, 메모리 사이즈 체크
print("train : ", trn.shape)
print("test  : ", tst.shape)
#print(trn.info())  #--> 4.9G (리눅스에서 읽어들일때 보다 파일이 두배는 크다)
#print(tst.info())  #--> 170M

train :  (13647309, 48)
test  :  (929615, 24)


검증용 데이터는 상품여부(타겟변수)가 없다. 나중에 훈련데이터와 합친 후 결측값은 0으로 다 채운다.

In [4]:
print("train : ", trn.shape) 
trn.tail()

train :  (13647309, 48)


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
13647304,2016-05-28,1166765,N,ES,V,22,2013-08-14,0.0,33,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
13647305,2016-05-28,1166764,N,ES,V,23,2013-08-14,0.0,33,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
13647306,2016-05-28,1166763,N,ES,H,47,2013-08-14,0.0,33,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
13647307,2016-05-28,1166789,N,ES,H,22,2013-08-14,0.0,33,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
13647308,2016-05-28,1550586,N,ES,H,37,2016-05-13,1.0,0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


---
### 데이터 전처리  

- 금융상품 결측값을 0으로 대체한다. 금융상품 보유 정보가 없으면 보유하고 있지 않다고 인정
- 훈련 데이터와 테스트 데이터를 통합한다(날짜 기준, feha_dato). 고객프로파일 변수 24개는 공유하나, 테스트에 없는 24개 금융상품 변수는 0으로 채운다.
- 범주형, 수치형 데이터를 전처리한다.
  * 범주형은 factorize()으로 label Encoding  
  * 데이터 타입이 object인 수치형 데이터는 .unique()로 특이값을 대체하거나 제거하고, 메모리를 적게 차지하는 정수형(np.int8)로 전환한다.
- 추후 모델학습에 사용할 변수 이름을 features 리스트에 미리 담는다.

---

In [86]:
# [참고] float, int 데이터 메모리 사이즈 체크 ... (1)
ser_uniform = pd.Series(np.random.uniform(0, 10, 10000)) # 0~10사이의 난수를 float로 생성
f_msize = ser_uniform.memory_usage()      
i_msize = ser_uniform.astype(np.int8).memory_usage()     # 정수형으로 변환
m_gap = (f_msize-i_msize)/f_msize * 100

print("메모리 점유율 : 실수형=>{0:,}byte, 정수형=>{1:,}byte, 메모리절감율=>{2:4.1f}%".format(f_msize, i_msize, m_gap))

메모리 점유율 : 실수형=>80,080byte, 정수형=>10,080byte, 메모리절감율=>87.4%


In [87]:
# [참고] float, int 데이터 메모리 사이즈 체크 ... (2)
dtypes = ['int8', 'float64']
data = dict([(t, np.ones(shape=2000).astype(t)) for t in dtypes])
pd.DataFrame(data).memory_usage()

Index         80
int8        2000
float64    16000
dtype: int64

##### 데이터 형식을 똑똑하게 정의하면, 메모리를 최대 90% 가까이 절약할 수 있다.  
---

In [4]:
# 타겟변수가 되는 금융상품 변수를 별도로 저장해 놓는다.
prods = trn.columns[24:].tolist()
prods

['ind_ahor_fin_ult1',
 'ind_aval_fin_ult1',
 'ind_cco_fin_ult1',
 'ind_cder_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_deco_fin_ult1',
 'ind_deme_fin_ult1',
 'ind_dela_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_viv_fin_ult1',
 'ind_nomina_ult1',
 'ind_nom_pens_ult1',
 'ind_recibo_ult1']

In [5]:
# 금융상품 변수 결측값을 미리 0으로 대체한다.
# 메모리를 절약하기 위해 정수형으로 저장한다. 
trn[prods] = trn[prods].fillna(0).astype(np.int8)

In [ ]:
# [***꿀팁***] 24개 금융상품 중 하나도 보유하지 않는 고객 데이터는 제거한다.
# 행(row)별로 금융상품 컬럼을 sum함수로 계산하고, 0인 경우를 bool로 처리
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

In [7]:
print("train : ", trn.shape)
trn.tail()

train :  (11091070, 48)


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
13647304,2016-05-28,1166765,N,ES,V,22,2013-08-14,0.0,33,1.0,...,0,0,0,0,0,0,0,0,0,0
13647305,2016-05-28,1166764,N,ES,V,23,2013-08-14,0.0,33,1.0,...,0,0,0,0,0,0,0,0,0,0
13647306,2016-05-28,1166763,N,ES,H,47,2013-08-14,0.0,33,1.0,...,0,0,0,0,0,0,0,0,0,0
13647307,2016-05-28,1166789,N,ES,H,22,2013-08-14,0.0,33,1.0,...,0,0,0,0,0,0,0,0,0,0
13647308,2016-05-28,1550586,N,ES,H,37,2016-05-13,1.0,0,1.0,...,0,0,0,0,0,0,0,0,0,0


#### 학습데이터에서 금융상품 신규매수가 없는 경우를 제외하였더니 13,647,309건에서 11,091,070건으로 줄었다. 

---
### 훈련데이터와 테스트 데이터를 통합하고, 범주형 변수는 label encogind한다

`# 참고 : pandas factorize 함수(na_sentinel=-99은 결측치 처리)  
labels, uniques = pd.factorize(['b', None, 'a', 'c', 'b'])  
labels
-> array([ 0, -1,  1,  2,  0])
uniques
-> array(['b', 'a', 'c'], dtype=object)`

In [6]:
# 테스트 데이터에 없는 금융상품 변수는 0으로 대체한다.
for col in trn.columns[24:]:
    tst[col] = 0
    
# 훈련데이터와 테스트데이터를 합친 df를 만든다.     
df = pd.concat([trn, tst], axis=0)

In [7]:
# 학습에 사용할 변수를 담는 list
features = []

# 범주형 변수를 .factorize() 함수로 label encoding 한다.
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi',
                    'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)  # 결측은 -99처리
features += categorical_cols

의문점!!!! 왜 -99로 대체할까? 

In [10]:
df.tail()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
929610,2016-06-28,660237,0,0,1,55,1999-04-21,0.0,206,1.0,...,0,0,0,0,0,0,0,0,0,0
929611,2016-06-28,660238,0,0,1,30,2006-11-29,0.0,115,1.0,...,0,0,0,0,0,0,0,0,0,0
929612,2016-06-28,660240,0,0,1,52,2006-11-29,0.0,115,1.0,...,0,0,0,0,0,0,0,0,0,0
929613,2016-06-28,660243,0,0,1,32,2006-11-29,0.0,115,1.0,...,0,0,0,0,0,0,0,0,0,0
929614,2016-06-28,660248,0,0,1,92,2006-11-29,0.0,115,1.0,...,0,0,0,0,0,0,0,0,0,0


#### [참고 ] Factorize 결과 확인

In [32]:
# factorize 처리 전 데이터
trn['ind_empleado'].value_counts() 

N    2764552
B        755
A        611
F        575
S          7
Name: ind_empleado, dtype: int64

In [31]:
# factorize 처리 후 데이터(trn+tst라 건수가 trn보다 많음에 유의)
df['ind_empleado'].value_counts()

 0     3693648
-99       5558
 1         973
 2         759
 3         727
 4           8
Name: ind_empleado, dtype: int64

In [8]:
# 수치형 변수의 특이값과 결측값을 -99로 대체하고, 정수형으로 변환한다.
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8) 

df['antiguedad'].replace('     NA',-99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

In [42]:
# 분포를 찍어보자~! 
varlists = ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

for var in varlists:
    print(df[var].describe().astype(np.int8))
    print("\n")

count    -87
mean      39
std       17
min      -99
25%       24
50%       38
75%       50
max      127
Name: age, dtype: int8


count    -87
mean       9
std       65
min     -128
25%      -37
50%       21
75%       47
max      127
Name: antiguedad, dtype: int8


count    -87
mean     -20
std       77
min     -128
25%      -99
50%      -34
75%       47
max      127
Name: renta, dtype: int8


count   -13
mean      0
std       0
min       0
25%       0
50%       0
75%       0
max       1
Name: ind_nuevo, dtype: int8


count   -13
mean      1
std       2
min       1
25%       1
50%       1
75%       1
max      99
Name: indrel, dtype: int8


count   -87
mean      0
std       7
min     -99
25%       1
50%       1
75%       1
max       5
Name: indrel_1mes, dtype: int8


count   -13
mean      0
std       0
min       0
25%       0
50%       1
75%       1
max       1
Name: ind_actividad_cliente, dtype: int8




**의문점!!! count가 왜 마이너스가 나올까?**

In [101]:
# 마지막에 생성되는 trn 데이터세트로 돌려보니... count 여전히 이상
trn['age'].describe().astype(np.int8)

count      6
mean      39
std       16
min        2
25%       24
50%       37
75%       50
max      117
Name: age, dtype: int8

In [105]:
trn['age'].describe()

count    3.462918e+06
mean     3.932476e+01
std      1.693281e+01
min      2.000000e+00
25%      2.400000e+01
50%      3.700000e+01
75%      5.000000e+01
max      1.170000e+02
Name: age, dtype: float64

In [106]:
# count만 따로 출력해보면...
trn['age'].describe().iloc[0]  # -> 제대로 나온다. 

3462918.0

count를 int로 변환하는 과정에서 describe함수가 정확한 값을 출력해주지 않는다. 

---

In [9]:
# 학습에 사용할 수치형 변수를 features에 추가한다.
features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

In [10]:
# 두 날짜 변수에서 연도와 월정보를 추출한다.
# (예) 2015-01-12 -> 연도와 월 추출
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float 
                                                              else float(x.split('-')[1])).astype(np.int8)

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float 
                                                              else float(x.split('-')[1])).astype(np.int8)

df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float 
                                                             else float(x.split('-')[0])).astype(np.int8)

In [14]:
df['fecha_alta'][:3]

0    2015-01-12
1    2012-08-10
2    2012-08-10
Name: fecha_alta, dtype: object

In [15]:
df['fecha_alta_month'][:3]

0    1
1    8
2    8
Name: fecha_alta_month, dtype: int8

In [16]:
df['ult_fec_cli_1t'][:3]   # ----> 대부분 데이터가 NaN -> 0으로 값 변환... 어떤 의미가 있을까? 

0    NaN
1    NaN
2    NaN
Name: ult_fec_cli_1t, dtype: object

In [17]:
df['ult_fec_cli_1t_month'][:3]  

0    0
1    0
2    0
Name: ult_fec_cli_1t_month, dtype: int8

In [18]:
df['ult_fec_cli_1t_month'].value_counts()

0     12013847
7         4387
6         1683
2           98
9           92
11          85
10          84
3           83
12          78
1           72
5           70
8           57
4           49
Name: ult_fec_cli_1t_month, dtype: int64

In [19]:
df['ult_fec_cli_1t_year'].value_counts()

 0     12013847
-33        4783
-32        2055
Name: ult_fec_cli_1t_year, dtype: int64

In [11]:
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']
##### =====> 교재에는 피처에 포함하지만, 대부분 값이 결측이므로 빼는것이 타당하지 않을까? 

In [12]:
# 그 외 변수의 결측값은 모두 -99로 대체한다.
df.fillna(-99, inplace=True)

---

### (피처엔지니어링) lag-1 데이터를 생성한다

학습용 데이터가 5개월치가 이력을 가지고 있으므로 lag-5까지 가능하다.  
baseline model에서는 lag-1만 사용한다.

In [13]:
# lag 기준 날짜(매우 중요) : lag된 데이터와 원본 데이터 merge시 고객번호와 함께 key값이 된다
def date_to_int(str_date):
    '''
    날짜를 숫자로 변환하는 함수이다. 
    2015-01-28은 1, 2016-06-28은 18로 변환한다.
    '''
    Y, M, D = [int(a) for a in str_date.strip().split('-')]
    int_date = (int(Y) - 2015)*12 + int(M)
    return int_date

In [14]:
# 날짜를 숫자로 변환하여 int_date에 저장한다. 
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

In [24]:
value_counts_add_percent(df['int_date'])

,count,percent
18,929615,7.733461
17,696539,5.794503
16,695044,5.782066
15,693270,5.767309
14,690235,5.742060
13,687830,5.722053
12,683915,5.689484
11,680043,5.657273
10,671324,5.584740
9,654328,5.443350


### lag 데이터 만들기

In [15]:
# 1. 데이터를 복사하고
df_lag = df.copy()

# 2. 변수명에 _prev를 추가한다.(key가 e고객번호 ncodpers와 날짜 int_date 제외)
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col 
                                for col in df.columns]
# 3 int_date 날짜 1을 더하여 lag을 생성한다
df_lag['int_date'] += 1

# 4. 원본 데이터와 lag데이터를 ncodper와 int_date 기준으로 합친다. 
# lag 데이터의 int_date는 1 밀려있기 때문에 이전 달의 정보가 삽입된다.
df_trn = df.merge(df_lag, on=['ncodpers', 'int_date'], how='left')

In [26]:
# (참고) lag이 적용되면 1이 증가한다. 
print(df['int_date'][:10].tolist())
print('==>')
print(df_lag['int_date'][:10].tolist())

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
==>
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [18]:
df_trn.tail()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_reca_fin_ult1_prev,ind_tjcr_fin_ult1_prev,ind_valo_fin_ult1_prev,ind_viv_fin_ult1_prev,ind_nomina_ult1_prev,ind_nom_pens_ult1_prev,ind_recibo_ult1_prev,fecha_alta_month_prev,ult_fec_cli_1t_month_prev,ult_fec_cli_1t_year_prev
12020680,2016-06-28,660237,0,0,1,55,1999-04-21,0.0,-50,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,4.0,0.0,0.0
12020681,2016-06-28,660238,0,0,1,30,2006-11-29,0.0,115,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12020682,2016-06-28,660240,0,0,1,52,2006-11-29,0.0,115,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0
12020683,2016-06-28,660243,0,0,1,32,2006-11-29,0.0,115,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0
12020684,2016-06-28,660248,0,0,1,92,2006-11-29,0.0,115,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0


In [19]:
df_trn.columns.tolist()

['fecha_dato',
 'ncodpers',
 'ind_empleado',
 'pais_residencia',
 'sexo',
 'age',
 'fecha_alta',
 'ind_nuevo',
 'antiguedad',
 'indrel',
 'ult_fec_cli_1t',
 'indrel_1mes',
 'tiprel_1mes',
 'indresi',
 'indext',
 'conyuemp',
 'canal_entrada',
 'indfall',
 'tipodom',
 'cod_prov',
 'nomprov',
 'ind_actividad_cliente',
 'renta',
 'segmento',
 'ind_ahor_fin_ult1',
 'ind_aval_fin_ult1',
 'ind_cco_fin_ult1',
 'ind_cder_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_deco_fin_ult1',
 'ind_deme_fin_ult1',
 'ind_dela_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_viv_fin_ult1',
 'ind_nomina_ult1',
 'ind_nom_pens_ult1',
 'ind_recibo_ult1',
 'fecha_alta_month',
 'ult_fec_cli_1t_month',
 'ult_fec_cli_1t_year',
 'int_date',
 'fecha_dato_prev',
 'ind_empleado_prev',
 'pais_residenc

In [ ]:
# 불필요한 데이터를 메모리에서 제거한다.
del df, df_lag

In [21]:
# 지난 달의 금융상품 정보가 존재하지 않을 경우에 대비하여 0으로 대체한다. 
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
    
df_trn.fillna(-99, inplace=True)

In [22]:
# feature 리스트에 lag-1변수를 추가한다.
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

features 리스트에 lag-1 함수를 추가했다는 것은  
**모델 입력 변수**로 들어간다는 의미이다. 

In [ ]:
###
### Baseline 모델 이후, 다양한 피처를 여기에 추가한다. 
###

---

### 머신러닝 모델학습 : 교차검증  

- 데이트를 훈련, 테스트용으로 분리한다.
- 학습에는 2016-01-28 ~ 2016-04-28만 사용하고, 검증에는 2016-05-28 데이터를 사용한다.

In [23]:
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
del df_trn

In [24]:
# 훈련 데이터에서 신규 구매건수만 추출한다.
X = []
Y = []

for i, prod in enumerate(prods):
    prev = prod + '_prev'    
    ##### 금융상품 피처 하나씩 pandas가 돌아간다. _start   #####
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]   # ===> lag으로 신규구매 찾기: 금융상품별 0->1인 것만 prX에 저장
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i  # ===> 신규구매 건수만큼, prods의 자릿수(금융상품 컬럼순서)를 채워넣는다. 
    ##### 금융상품 피처 하나씩 pandas가 돌아간다. _end     #####
    X.append(prX)
    Y.append(prY)
    
XY = pd.concat(X)   # 리스트에 금융상품 컬럼별 신규구매 데이터가 pandas dataframe으로 들어있다. 
Y = np.hstack(Y)    # X의 총 데이터수에 매칭되는 금융상품 컬럼번호 저장
XY['y'] = Y         # 타겟변수로 추가한다

In [25]:
# X는? 24개 변수별로 신규 구매건의 데이터프레임을 리스트로 저장하는 독특한 방식
for i in range(len(X)):
    print("col : {} -> {}".format(prods[i], X[i].shape))

col : ind_ahor_fin_ult1 -> (1, 102)
col : ind_aval_fin_ult1 -> (1, 102)
col : ind_cco_fin_ult1 -> (33164, 102)
col : ind_cder_fin_ult1 -> (37, 102)
col : ind_cno_fin_ult1 -> (11725, 102)
col : ind_ctju_fin_ult1 -> (424, 102)
col : ind_ctma_fin_ult1 -> (4409, 102)
col : ind_ctop_fin_ult1 -> (1167, 102)
col : ind_ctpp_fin_ult1 -> (713, 102)
col : ind_deco_fin_ult1 -> (1, 102)
col : ind_deme_fin_ult1 -> (0, 102)
col : ind_dela_fin_ult1 -> (1578, 102)
col : ind_ecue_fin_ult1 -> (11549, 102)
col : ind_fond_fin_ult1 -> (467, 102)
col : ind_hip_fin_ult1 -> (18, 102)
col : ind_plan_fin_ult1 -> (226, 102)
col : ind_pres_fin_ult1 -> (32, 102)
col : ind_reca_fin_ult1 -> (2316, 102)
col : ind_tjcr_fin_ult1 -> (20443, 102)
col : ind_valo_fin_ult1 -> (1550, 102)
col : ind_viv_fin_ult1 -> (18, 102)
col : ind_nomina_ult1 -> (26392, 102)
col : ind_nom_pens_ult1 -> (30761, 102)
col : ind_recibo_ult1 -> (52042, 102)


In [26]:
X[0]  # 리스트 첫번째 요소가 데이터프레임이다 ㄷㄷㄷㄷ

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_reca_fin_ult1_prev,ind_tjcr_fin_ult1_prev,ind_valo_fin_ult1_prev,ind_viv_fin_ult1_prev,ind_nomina_ult1_prev,ind_nom_pens_ult1_prev,ind_recibo_ult1_prev,fecha_alta_month_prev,ult_fec_cli_1t_month_prev,ult_fec_cli_1t_year_prev
10597872,2016-05-28,194160,0,0,0,42,2000-09-25,0.0,-68,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0


In [28]:
print(len(X[2]))
X[2].head()

33164


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_reca_fin_ult1_prev,ind_tjcr_fin_ult1_prev,ind_valo_fin_ult1_prev,ind_viv_fin_ult1_prev,ind_nomina_ult1_prev,ind_nom_pens_ult1_prev,ind_recibo_ult1_prev,fecha_alta_month_prev,ult_fec_cli_1t_month_prev,ult_fec_cli_1t_year_prev
7628180,2016-01-28,1432311,0,0,1,26,2015-08-07,1.0,5,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0
7628198,2016-01-28,1432232,0,0,1,33,2015-08-07,0.0,19,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0
7628482,2016-01-28,1432080,0,0,0,23,2015-08-07,1.0,5,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0
7628692,2016-01-28,1432952,0,0,0,77,2015-08-10,1.0,5,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
7628745,2016-01-28,1432622,0,0,1,21,2015-08-10,1.0,5,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0


In [29]:
# 그렇다면 Y는...?  --> 어떤 상품을 신규구매했는지 컬럼 번호 붙어있음(X와 대칭)
print(Y.shape)
Y

(199034,)


array([ 0,  1,  2, ..., 23, 23, 23], dtype=int8)

In [30]:
value_counts_add_percent(pd.Series(Y))

,count,percent
23,52042,26.147291
2,33164,16.662480
22,30761,15.455148
21,26392,13.260046
18,20443,10.271109
4,11725,5.890953
12,11549,5.802526
6,4409,2.215199
17,2316,1.163620
11,1578,0.792829


In [31]:
# 그렇다면 XY는 ..? 
# ===> 신규구매 X의 모든 데이터를 합쳤다. Y는 해당 금융상품 
XY.shape

(199034, 103)

In [32]:
XY.tail() # --> reset_index 안 해도 되나?????

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_tjcr_fin_ult1_prev,ind_valo_fin_ult1_prev,ind_viv_fin_ult1_prev,ind_nomina_ult1_prev,ind_nom_pens_ult1_prev,ind_recibo_ult1_prev,fecha_alta_month_prev,ult_fec_cli_1t_month_prev,ult_fec_cli_1t_year_prev,y
11090455,2016-05-28,1166385,0,0,1,65,2013-08-14,0.0,33,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,23
11090468,2016-05-28,1166355,0,0,1,30,2013-08-14,0.0,33,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,23
11090667,2016-05-28,1166343,0,0,1,36,2013-08-14,0.0,33,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,23
11090696,2016-05-28,1166232,0,0,0,23,2013-08-14,0.0,33,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,8.0,0.0,0.0,23
11090782,2016-05-28,1166874,0,0,1,22,2013-08-16,0.0,33,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,23


---

In [34]:
# 훈련, 검증 데이터로 분리한다. 
vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]

In [35]:
# XGBoost 모델 parameter를 설정한다.
### 튜토리얼 https://xgboost.readthedocs.io/en/latest/python/index.html
param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 4,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018,
    }

### XGBoost 파라미터  
- max_depth : 트리 모델의 최대 깊이. 값이 클수록 더 복잡한 모델 생성하며, 과적합될 수 있음  
- eta : 딥러닝의 learning rate와 같은 개념. 0과 1사이의 값을 가짐. 값이 크면 학습이 잘 되지 않을 수 있으며, 값이 작으면 느림  
- colsample_bytree : 트리 생성시 훈련데이터에서 변수 샘플링 비율. 모든 트리는 전체 변수의 일부만 학습하여 서로의 약점을 보완. 보통 0.6~0.9 사이의 값을 사용  
- colsample_bylevel : 트리 레벨별로 훈련 데이터의 변수 샘플링 비율. 보통 0.6~0.9사이의 값을 사용

In [36]:
# 훈련, 검증 데이터를 XGBoost 형태로 변환한다.
X_trn = XY_trn.as_matrix(columns=features)   # ---> features만 트레이닝셑의 행렬로 저장함
Y_trn = XY_trn.as_matrix(columns=['y'])
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

X_vld = XY_vld.as_matrix(columns=features)
Y_vld = XY_vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [39]:
print(len(features))
X_trn.shape   # ---> features에 저장된 컬럼만 training set으로 

66


(161137, 66)

In [37]:
features

['ind_empleado',
 'pais_residencia',
 'sexo',
 'tiprel_1mes',
 'indresi',
 'indext',
 'conyuemp',
 'canal_entrada',
 'indfall',
 'tipodom',
 'nomprov',
 'segmento',
 'age',
 'antiguedad',
 'renta',
 'ind_nuevo',
 'indrel',
 'indrel_1mes',
 'ind_actividad_cliente',
 'ult_fec_cli_1t_month',
 'ult_fec_cli_1t_year',
 'ind_empleado_prev',
 'pais_residencia_prev',
 'sexo_prev',
 'tiprel_1mes_prev',
 'indresi_prev',
 'indext_prev',
 'conyuemp_prev',
 'canal_entrada_prev',
 'indfall_prev',
 'tipodom_prev',
 'nomprov_prev',
 'segmento_prev',
 'age_prev',
 'antiguedad_prev',
 'renta_prev',
 'ind_nuevo_prev',
 'indrel_prev',
 'indrel_1mes_prev',
 'ind_actividad_cliente_prev',
 'ult_fec_cli_1t_month_prev',
 'ult_fec_cli_1t_year_prev',
 'ind_ahor_fin_ult1_prev',
 'ind_aval_fin_ult1_prev',
 'ind_cco_fin_ult1_prev',
 'ind_cder_fin_ult1_prev',
 'ind_cno_fin_ult1_prev',
 'ind_ctju_fin_ult1_prev',
 'ind_ctma_fin_ult1_prev',
 'ind_ctop_fin_ult1_prev',
 'ind_ctpp_fin_ult1_prev',
 'ind_deco_fin_ult1_prev',

In [40]:
XY_trn[features].iloc[0]

ind_empleado                   0.0
pais_residencia                0.0
sexo                           1.0
tiprel_1mes                    0.0
indresi                        0.0
indext                         0.0
conyuemp                     -99.0
canal_entrada                  3.0
indfall                        0.0
tipodom                        0.0
nomprov                       18.0
segmento                       2.0
age                           43.0
antiguedad                     3.0
renta                        -99.0
ind_nuevo                      1.0
indrel                         1.0
indrel_1mes                    1.0
ind_actividad_cliente          1.0
ult_fec_cli_1t_month           0.0
ult_fec_cli_1t_year            0.0
ind_empleado_prev              0.0
pais_residencia_prev           0.0
sexo_prev                      1.0
tiprel_1mes_prev               0.0
indresi_prev                   0.0
indext_prev                    0.0
conyuemp_prev                -99.0
canal_entrada_prev  

In [41]:
X_trn[0,:]

array([  0.,   0.,   1.,   0.,   0.,   0., -99.,   3.,   0.,   0.,  18.,
         2.,  43.,   3., -99.,   1.,   1.,   1.,   1.,   0.,   0.,   0.,
         0.,   1.,   0.,   0.,   0., -99.,   3.,   0.,   0.,  18.,   2.,
        43.,   2., -99.,   1.,   1.,   1.,   1.,   0.,   0.,   0.,   0.,
         1.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   1.,   1.,   0.,   0.,   0.,   0.,   1.])

In [42]:
dtrn  # 제너레이터로 저장

In [43]:
# XGBoost 모델을 훈련 데이터로 학습한다!
print('--------------------------> start')
print(datetime.datetime.now(timezone('Asia/Seoul')))
print('\n')

watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)

print('--------------------------> end')
print(datetime.datetime.now(timezone('Asia/Seoul')))

--------------------------> start
2018-10-15 17:42:17.144485+09:00


[0]	train-mlogloss:2.69601	eval-mlogloss:2.70578
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:2.46902	eval-mlogloss:2.48286
[2]	train-mlogloss:2.28773	eval-mlogloss:2.30235
[3]	train-mlogloss:2.15299	eval-mlogloss:2.16842
[4]	train-mlogloss:2.04977	eval-mlogloss:2.0655
[5]	train-mlogloss:1.95953	eval-mlogloss:1.97571
[6]	train-mlogloss:1.88028	eval-mlogloss:1.8977
[7]	train-mlogloss:1.81328	eval-mlogloss:1.83151
[8]	train-mlogloss:1.75027	eval-mlogloss:1.7689
[9]	train-mlogloss:1.69514	eval-mlogloss:1.71352
[10]	train-mlogloss:1.64538	eval-mlogloss:1.66382
[11]	train-mlogloss:1.60118	eval-mlogloss:1.61988
[12]	train-mlogloss:1.56174	eval-mlogloss:1.5809
[13]	train-mlogloss:1.52612	eval-mlogloss:1.5453
[14]	train-mlogloss:1.49144	eval-mlogloss:1.51069
[15]	train-mlogloss:1.46035	eval-mlogloss:1.4

[160]	train-mlogloss:1.01243	eval-mlogloss:1.08883
[161]	train-mlogloss:1.01199	eval-mlogloss:1.08879
[162]	train-mlogloss:1.01132	eval-mlogloss:1.08878
[163]	train-mlogloss:1.01083	eval-mlogloss:1.08874
[164]	train-mlogloss:1.01034	eval-mlogloss:1.08872
[165]	train-mlogloss:1.00989	eval-mlogloss:1.08869
[166]	train-mlogloss:1.00933	eval-mlogloss:1.08867
[167]	train-mlogloss:1.00891	eval-mlogloss:1.08861
[168]	train-mlogloss:1.00854	eval-mlogloss:1.08858
[169]	train-mlogloss:1.00806	eval-mlogloss:1.08856
[170]	train-mlogloss:1.00762	eval-mlogloss:1.08854
[171]	train-mlogloss:1.0072	eval-mlogloss:1.08857
[172]	train-mlogloss:1.00664	eval-mlogloss:1.08856
[173]	train-mlogloss:1.0061	eval-mlogloss:1.08853
[174]	train-mlogloss:1.00558	eval-mlogloss:1.0885
[175]	train-mlogloss:1.00508	eval-mlogloss:1.08847
[176]	train-mlogloss:1.00458	eval-mlogloss:1.08838
[177]	train-mlogloss:1.00406	eval-mlogloss:1.08839
[178]	train-mlogloss:1.00356	eval-mlogloss:1.08839
[179]	train-mlogloss:1.00293	eval-

컴 기준으로 40분 소요

In [44]:
# 학습한 모델을 저장한다.
import pickle
pickle.dump(model, open("model/xgb.baseline.pkl", "wb"))
best_ntree_limit = model.best_ntree_limit

### kaggle에서 요구한 평가 함수 : map@7

In [45]:
#### MAP@7 함수 #### 
def apk(actual, predicted, k=7, default=0.0):
    # MAP@7 이므로, 최대 7개만 사용한다
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        # 점수를 부여하는 조건은 다음과 같다 :
        # 예측값이 정답에 있고 (‘p in actual’)
        # 예측값이 중복이 아니면 (‘p not in predicted[:i]’) 
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    # 정답값이 공백일 경우, 무조건 0.0점을 반환한다
    if not actual:
        return default

    # 정답의 개수(len(actual))로 average precision을 구한다
    return score / min(len(actual), k)

def mapk(actual, predicted, k=7, default=0.0):
    # list of list인 정답값(actual)과 예측값(predicted)에서 고객별 Average Precision을 구하고, np.mean()을 통해 평균을 계산한다
    return np.mean([apk(a, p, k, default) for a, p in zip(actual, predicted)]) 

In [46]:
# MPA@7평가 척도를 위한 준비작업
# 고객식별번호를 추출한다.
vld = trn[trn['fecha_dato'] == vld_date]
ncodpers_vld = vld.as_matrix(columns=['ncodpers'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [47]:
ncodpers_vld.shape

(696539, 1)

In [48]:
ncodpers_vld[:10]

array([[657788],
       [657795],
       [657790],
       [657794],
       [657789],
       [657787],
       [657777],
       [657781],
       [657780],
       [657779]], dtype=int64)

In [ ]:
# vld 검증 데이터에서 신규 구매를 구한다.
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    vld[padd] = vld[prod] - vld[prev]    
    
add_vld = vld.as_matrix(columns=[prod + '_add' for prod in prods])  # 행렬에 결과값은 비어있다. 
add_vld_list = [list() for i in range(len(ncodpers_vld))]           # 리스트에 결과값은 비어있다. 

In [50]:
add_vld.shape

(696539, 24)

In [51]:
add_vld[:10,:]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.

In [52]:
add_vld_list[:10]

[[], [], [], [], [], [], [], [], [], []]

In [53]:
# 고객별 신규 구매 정답 값을 add_vld_list에 저장하고, 총 count를 count_vld에 저장한다.
# Dummy 작업 ????
count_vld = 0
for ncodper in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodper, prod] > 0:
            add_vld_list[ncodper].append(prod)
            count_vld += 1

In [54]:
add_vld_list[:5]

[[], [], [], [], []]

In [55]:
# 검증 데이터에서 얻을 수 있는 MAP@7 최고점을 미리 구한다. (0.042663)
print(mapk(add_vld_list, add_vld_list, 7, 0.0))

0.0426637991555


In [56]:
len(add_vld_list)

696539

In [59]:
# 검증 데이터에 대한 예측 값을 구한다.
X_vld = vld.as_matrix(columns=features)
Y_vld = vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [60]:
preds_vld.shape

(696539, 24)

In [61]:
preds_vld[:3,:]

array([[  1.79764208e-06,   6.50783795e-06,   4.33127243e-05,
          1.91678206e-04,   1.58431530e-02,   8.53669098e-06,
          1.94710528e-03,   1.63869429e-02,   1.68497600e-02,
          6.50783795e-06,   1.79764208e-06,   9.15754214e-03,
          9.75837782e-02,   9.38877114e-04,   2.59925782e-05,
          1.32443255e-03,   2.43254945e-05,   8.57609231e-03,
          1.28952518e-01,   3.97941424e-03,   5.02492003e-05,
          3.07504945e-02,   3.06793209e-02,   6.36669874e-01],
       [  2.15568843e-06,   7.80404025e-06,   4.29051579e-05,
          6.17192069e-04,   1.36247296e-02,   1.00677626e-05,
          3.85553984e-04,   6.01040293e-03,   2.68579950e-03,
          7.80404025e-06,   2.15568843e-06,   1.86876226e-02,
          5.08233830e-02,   7.42097571e-03,   2.57453648e-05,
          4.39618714e-04,   4.69803927e-05,   1.15746204e-02,
          1.64465681e-01,   1.09242592e-02,   8.77524290e-05,
          3.27937417e-02,   2.81337537e-02,   6.51179254e-01],
      

In [62]:
# 이전 달에 보유한 제품은 신규 구매가 불가하기 때문에, 확률값에서 미리 1을 빼준다
preds_vld = preds_vld - vld.as_matrix(columns=[prod + '_prev' for prod in prods])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [63]:
# 검증 데이터 예측 상위 7개를 추출한다.
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result_vld.append([ip for y,p,ip in y_prods])

In [64]:
print(len(result_vld))
result_vld[:4]

696539


[[23, 18, 12, 21, 22, 8, 7],
 [23, 18, 12, 21, 22, 11, 4],
 [8, 17, 2, 19, 7, 15, 11],
 [23, 18, 12, 7, 13, 22, 21]]

In [65]:
# 검증 데이터에서의 MAP@7 점수를 구한다. (0.036466)
print(mapk(add_vld_list, result_vld, 7, 0.0))

0.0364348866095


In [110]:
# 풀지못한 숙제 .... add_vld_list는 비어있는데... 
add_vld_list[:10] 

[[], [], [], [], [], [], [], [], [], []]

---

### 피처 중요도를 구해보자. 
XGBoost 모델을 전체 훈련 데이터로 재학습한다!

In [66]:
X_all = XY.as_matrix(columns=features)
Y_all = XY.as_matrix(columns=['y'])
dall = xgb.DMatrix(X_all, label=Y_all, feature_names=features)
watch_list = [(dall, 'train')]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [67]:
# 트리 개수를 늘어난 데이터 양만큼 비례해서 증가한다.
best_ntree_limit = int(best_ntree_limit * (len(XY_trn) + len(XY_vld)) / len(XY_trn))
# XGBoost 모델 재학습!
model = xgb.train(param, dall, num_boost_round=best_ntree_limit, evals=watch_list)

[0]	train-mlogloss:2.67839
[1]	train-mlogloss:2.43976
[2]	train-mlogloss:2.27317
[3]	train-mlogloss:2.14291
[4]	train-mlogloss:2.02787
[5]	train-mlogloss:1.94036
[6]	train-mlogloss:1.85989
[7]	train-mlogloss:1.79227
[8]	train-mlogloss:1.73054
[9]	train-mlogloss:1.67496
[10]	train-mlogloss:1.6265
[11]	train-mlogloss:1.58374
[12]	train-mlogloss:1.54474
[13]	train-mlogloss:1.50952
[14]	train-mlogloss:1.47667
[15]	train-mlogloss:1.44859
[16]	train-mlogloss:1.42192
[17]	train-mlogloss:1.39724
[18]	train-mlogloss:1.37631
[19]	train-mlogloss:1.35565
[20]	train-mlogloss:1.33703
[21]	train-mlogloss:1.32095
[22]	train-mlogloss:1.30465
[23]	train-mlogloss:1.2896
[24]	train-mlogloss:1.27635
[25]	train-mlogloss:1.2636
[26]	train-mlogloss:1.25174
[27]	train-mlogloss:1.24079
[28]	train-mlogloss:1.23086
[29]	train-mlogloss:1.22121
[30]	train-mlogloss:1.21248
[31]	train-mlogloss:1.20376
[32]	train-mlogloss:1.19583
[33]	train-mlogloss:1.18833
[34]	train-mlogloss:1.1815
[35]	train-mlogloss:1.17483
[36]	t

In [68]:
# 변수 중요도를 출력해본다. 예상하던 변수가 상위로 올라와 있는가?
print("Feature importance:")
for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
    print(kv)

Feature importance:
('renta', 25827)
('antiguedad', 25019)
('age', 23558)
('antiguedad_prev', 18492)
('age_prev', 16147)
('nomprov', 14134)
('renta_prev', 11106)
('canal_entrada', 10354)
('nomprov_prev', 8040)
('canal_entrada_prev', 5789)
('ind_recibo_ult1_prev', 4041)
('sexo', 4003)
('ind_ecue_fin_ult1_prev', 3560)
('ind_cco_fin_ult1_prev', 3475)
('ind_cno_fin_ult1_prev', 3219)
('segmento', 2617)
('ind_reca_fin_ult1_prev', 2505)
('ind_tjcr_fin_ult1_prev', 2479)
('segmento_prev', 2263)
('tiprel_1mes', 1866)
('ind_ctop_fin_ult1_prev', 1804)
('ind_valo_fin_ult1_prev', 1785)
('ind_nom_pens_ult1_prev', 1775)
('ind_dela_fin_ult1_prev', 1732)
('ind_nomina_ult1_prev', 1695)
('sexo_prev', 1653)
('ind_ctpp_fin_ult1_prev', 1424)
('ind_actividad_cliente', 1400)
('tiprel_1mes_prev', 1341)
('ind_fond_fin_ult1_prev', 1120)
('ind_actividad_cliente_prev', 1022)
('ind_ctma_fin_ult1_prev', 1021)
('ind_nuevo', 934)
('indext', 914)
('ind_plan_fin_ult1_prev', 696)
('ind_hip_fin_ult1_prev', 603)
('ind_nuevo

#### TOP 20 피처
('renta', 25827) : 가구 총수입  
('antiguedad', 25019) : 은행 누적 거래기간(월)  
('age', 23558) : 연령  
('antiguedad_prev', 18492) : 은행 누적 거래기간(월) lag-1  
('age_prev', 16147) : 연령 lag-1  
('nomprov', 14134) : 지방 이름(factorize, label encoding)  
('renta_prev', 11106) : 가구 총수익 lag-1  
('canal_entrada', 10354) : 고객 유입 채널  
('nomprov_prev', 8040) : 지방 이름 lag-1  
('canal_entrada_prev', 5789) : 고객 유입 채널  
('ind_recibo_ult1_prev', 4041) : 직불카드 거래여부  lag-1  
('sexo', 4003) : 성별  
('ind_ecue_fin_ult1_prev', 3560) : e-계정 거래여부 lag-1  
('ind_cco_fin_ult1_prev', 3475) : 당좌예금 거래여부 lag-1  
('ind_cno_fin_ult1_prev', 3219) : 급여계정 거래여부 lag-1  
('segmento', 2617) : 고객분류(VIP, 개인, 대졸-학자금대출?)  
('ind_reca_fin_ult1_prev', 2505) : 세금 lag-1  
('ind_tjcr_fin_ult1_prev', 2479) : 신용카드 거래여부 lag-1  

---
### Baseline 모델의 마지막 여정. 캐글 제출을 위하여 테스트 데이터에 대한 예측 값을 구한다.

In [69]:
X_tst = tst.as_matrix(columns=features)
dtst = xgb.DMatrix(X_tst, feature_names=features)
preds_tst = model.predict(dtst, ntree_limit=best_ntree_limit)

ncodpers_tst = tst.as_matrix(columns=['ncodpers'])
preds_tst = preds_tst - tst.as_matrix(columns=[prod + '_prev' for prod in prods])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [92]:
print(preds_tst.shape)
preds_tst[:3,:]

(929615, 24)


array([[  1.13725691e-05,   1.13725691e-05,  -9.99889271e-01,
          2.19470891e-03,   1.17776617e-02,   1.88844297e-05,
          6.22055028e-04,   1.26772400e-04,  -9.99619867e-01,
          1.13725691e-05,   1.73300612e-06,   6.50715875e-03,
          2.56516904e-01,   1.64239705e-02,   1.70405707e-04,
          1.54621666e-03,   3.73873772e-04,   2.34813746e-02,
         -9.99633452e-01,  -9.99122573e-01,   1.30338361e-04,
          2.44651679e-02,   2.94533819e-02,   6.24420404e-01],
       [  4.02542082e-06,   4.02542082e-06,  -9.99972275e-01,
          3.69612353e-05,   5.53940609e-02,   6.08684695e-06,
          8.42952868e-04,   4.36639712e-05,   6.90309535e-05,
          4.02542082e-06,   6.13412681e-07,   1.22994219e-03,
          1.98272932e-02,   1.26017621e-04,   1.59868650e-05,
          1.03820828e-04,   3.13265009e-05,   3.43917720e-02,
          1.13380770e-03,   4.00778692e-04,   9.82348047e-06,
          5.29003590e-02,   3.74452733e-02,   7.95950651e-01],
      

In [ ]:
# 캐글에서 정의한 양식에 맞추어 제출 파일을 생성한다.
# 파일과 헤더를 생성한다
submit_file = open('../model/xgb.baseline.2015-06-28', 'w')
submit_file.write('ncodpers,added_products\n')

In [94]:
# 고객번호마다 7개의 추천 금융상품 명을 넣는다. 
for ncodper, pred in zip(ncodpers_tst, preds_tst):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    y_prods = [p for y,p,ip in y_prods]
    submit_file.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))

In [98]:
# 제출 파일 눈으로 확인하기
pd.read_csv('model/xgb.baseline.2015-06-28', nrows=5)

,ncodpers,added_products
0,15889,ind_recibo_ult1 ind_ecue_fin_ult1 ind_nom_pens...
1,1170544,ind_recibo_ult1 ind_cno_fin_ult1 ind_nomina_ul...
2,1170545,ind_recibo_ult1 ind_cno_fin_ult1 ind_nom_pens_...
3,1170547,ind_recibo_ult1 ind_nomina_ult1 ind_cno_fin_ul...
4,1170548,ind_recibo_ult1 ind_nom_pens_ult1 ind_cno_fin_...


--- 

### 느낀 점

- 역시 캐글. 난이도도 상당하고 코드 작성 기술도 배울 점이 많다.
- 파이썬 기본 문법을 잘 이해하고 활용하자. lambda, map, sort, join, ...
- 메모리 효율적 관리 : 정수형 저장, 중간마다 불필요한 데이터 즉시 삭제
- 정말로 중요한 것 : 피처엔지니어링, 도메인 지식. 모델링 기법은 거들 뿐

---

### 비판적 사고

- 내가 알고 있다고 남도 아는 것은 아니다 : 책이 아마도 본인의 강의와 분석 경험을 그대로 옮겨 놓은 듯. 텍스트로만 접하는 독자에게 완결성은 상당히 부족하다.
- 코드를 함축적으로 짠다고 반드시 좋은 것인지? (파이썬 날코드, list comprehension) vs (Pandas, Numpy)
- 훈련, 검증 데이터 생성 과정을 간소화 할 수 없을까?

---

### 남아있는 숙제

- baseline model을 넘어서 -> 8등 소스코드 해석하기
- 데이터 세트 분리 엄청 복잡. 체계적으로 이해 필요 : trn, tst, lag, xy, x, y, ........ ㅠ.ㅠ
- 피처 엔지니어링 사례 추가 연구  
- multiclass 예측 방법론 연구 + binary 예측 모델을 여러 개 구성하는 것과 장단점
- 머신러닝 : xgboost, lgbm 공부. random forest 등 다른 모델과의 예측 성능 비교
- 과제별 모델성과 측정방법 연구 : (예) MAPK@7